In [1]:
import cv2
import os

Global variables

In [21]:
# folder path of the original images
folder_path = '/Users/naominorris/cps/original-photos'

# folders where the proccessed images are saved 
# noise reduction folders
output_noise_gaussian = '/Users/naominorris/cps/modified-photos/noise-reduction/gaussian'
output_noise_median = '/Users/naominorris/cps/modified-photos/noise-reduction/median'

# histogram equalization folders
output_hist_clahe = '/Users/naominorris/cps/modified-photos/histogram-equalization/clahe'
output_hist_standard = '/Users/naominorris/cps/modified-photos/histogram-equalization/standard'

# binarization folders
output_bi_global = '/Users/naominorris/cps/modified-photos/binarization/global'
output_bi_otsu = '/Users/naominorris/cps/modified-photos/binarization/otsu'
output_bi_adaptive = '/Users/naominorris/cps/modified-photos/binarization/adaptive'
output_bi_otsu_inv = '/Users/naominorris/cps/modified-photos/binarization/inv-otsu'

# morphological operations folders
output_morphological_opening = '/Users/naominorris/cps/modified-photos/morphological/opening'
output_morphological_closing = '/Users/naominorris/cps/modified-photos/morphological/closing'

# Edge detection folders
output_edge_unsharp = '/Users/naominorris/cps/modified-photos/edge-detection/unsharp-masking'
output_edge_sobel = '/Users/naominorris/cps/modified-photos/edge-detection/sobel'
output_edge_laplacian = '/Users/naominorris/cps/modified-photos/edge-detection/laplacian'  

load images

In [20]:
def load_images_from_folder(folder_path, extensions=('.png', '.jpg', '.jpeg', '.JPG')):
    
    # Check if the folder exists
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The specified folder does not exist: {folder_path}")

    # List comprehension to gather all image paths with the specified extensions
    image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(extensions)]

    return image_paths

## Noise Reduction

In [12]:
def apply_gaussian_blur(image_paths, output_folder, kernel_size=(5, 5), sigma=0):
    """
    Applies Gaussian blur to the input images and saves the results.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_path in image_paths:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error loading image {image_path}")
            continue

        blurred_img = cv2.GaussianBlur(img, kernel_size, sigma)
        output_path = os.path.join(output_folder, f"gaussian_{os.path.basename(image_path)}")
        cv2.imwrite(output_path, blurred_img)

## Histogram Equlalization

In [13]:
def apply_clahe(image_paths, output_folder, clip_limit=2.0, grid_size=(8, 8)):
    """
    Applies CLAHE (Contrast Limited Adaptive Histogram Equalization) to the input images.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size)

    for image_path in image_paths:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Error loading image {image_path}")
            continue

        cl1 = clahe.apply(img)
        output_path = os.path.join(output_folder, f'clahe_{os.path.basename(image_path)}')
        cv2.imwrite(output_path, cl1)

## Biniarzation

In [14]:
def apply_otsu_thresholding_inv(image_paths, output_folder):
    """
    Applies Otsu's thresholding with binary inversion to the input images.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_path in image_paths:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Error loading image {image_path}")
            continue

        _, binary_inv = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        output_path = os.path.join(output_folder, f"otsu_inv_{os.path.basename(image_path)}")
        cv2.imwrite(output_path, binary_inv)


In [19]:
def apply_otsu_thresholding(image_paths, output_folder):
    """
    Applies Otsu's thresholding to the input images.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_path in image_paths:
        # Read the image in grayscale
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Error loading image {image_path}")
            continue

        # Apply Otsu's thresholding
        _, th2 = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Save the thresholded image
        output_path = os.path.join(output_folder, f"otsu_{os.path.basename(image_path)}")
        cv2.imwrite(output_path, th2)


## Morophological Operations

In [15]:
def apply_morphological_closing(image_paths, output_folder, kernel_size=(5, 5)):
    """
    Applies morphological closing to the input images.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)

    for image_path in image_paths:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Error loading image {image_path}")
            continue

        closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
        output_path = os.path.join(output_folder, f"closing_{os.path.basename(image_path)}")
        cv2.imwrite(output_path, closing)


In [18]:
def apply_morphological_opening(image_paths, output_folder, kernel_size=(5, 5)):
    """
    Applies morphological opening to the input images.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)

    for image_path in image_paths:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Error loading image {image_path}")
            continue

        # Apply morphological opening
        opened_img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

        # Save the processed image
        output_path = os.path.join(output_folder, f"opened_{os.path.basename(image_path)}")
        cv2.imwrite(output_path, opened_img)
 


## Edge Decetion

In [16]:
def apply_sobel_edge_detection(image_paths, output_folder):
    """
    Applies Sobel edge detection to the input images.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_path in image_paths:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Error loading image {image_path}")
            continue

        sobel_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
        sobel_combined = cv2.magnitude(sobel_x, sobel_y)
        sobel_combined = cv2.convertScaleAbs(sobel_combined)

        output_path = os.path.join(output_folder, f"sobel_{os.path.basename(image_path)}")
        cv2.imwrite(output_path, sobel_combined)

In [22]:
def apply_unsharp_masking(image_paths, output_folder, sigma=1.0, strength=1.5):
    """
    Applies unsharp masking to the input images.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_path in image_paths:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Error loading image {image_path}")
            continue

        # Apply Gaussian blur to create the mask
        blurred = cv2.GaussianBlur(img, (0, 0), sigma)

        # Create the unsharp mask
        mask = cv2.subtract(img, blurred)

        # Enhance the image using the unsharp mask
        sharpened = cv2.addWeighted(img, 1 + strength, mask, strength, 0)

        # Save the sharpened image
        output_path = os.path.join(output_folder, f"unsharp_{os.path.basename(image_path)}")
        cv2.imwrite(output_path, sharpened)


## Image flows

Main Flow ending with Sobel

In [17]:
def process_image_flow_sobel(folder_path, output_noise_gaussian, output_hist_clahe, output_bi_otsu_inv, output_morphological_closing, output_edge_sobel):
    # Step 1: Load images
    image_paths = load_images_from_folder(folder_path)

    # Step 2: Apply Gaussian Blur
    apply_gaussian_blur(image_paths, output_noise_gaussian)

    # Step 3: Apply CLAHE
    blurred_image_paths = load_images_from_folder(output_noise_gaussian)
    apply_clahe(blurred_image_paths, output_hist_clahe)

    # Step 4: Apply Otsu Thresholding Inverted
    clahe_image_paths = load_images_from_folder(output_hist_clahe)
    apply_otsu_thresholding_inv(clahe_image_paths, output_bi_otsu_inv)

    # Step 5: Apply Morphological Closing
    otsu_image_paths = load_images_from_folder(output_bi_otsu_inv)
    apply_morphological_closing(otsu_image_paths, output_morphological_closing)

    # Step 6: Apply Sobel Edge Detection
    morph_image_paths = load_images_from_folder(output_morphological_closing)
    apply_sobel_edge_detection(morph_image_paths, output_edge_sobel)

Main flow ending with unsharp

In [ ]:
def process_image_flow_sharp(folder_path, output_noise_gaussian, output_hist_clahe, output_bi_otsu_inv, output_morphological_closing, output_edge_unsharp):
    # Step 1: Load images
    image_paths = load_images_from_folder(folder_path)

    # Step 2: Apply Gaussian Blur
    apply_gaussian_blur(image_paths, output_noise_gaussian)

    # Step 3: Apply CLAHE
    blurred_image_paths = load_images_from_folder(output_noise_gaussian)
    apply_clahe(blurred_image_paths, output_hist_clahe)

    # Step 4: Apply Otsu Thresholding Inverted
    clahe_image_paths = load_images_from_folder(output_hist_clahe)
    apply_otsu_thresholding_inv(clahe_image_paths, output_bi_otsu_inv)

    # Step 5: Apply Morphological Closing
    otsu_image_paths = load_images_from_folder(output_bi_otsu_inv)
    apply_morphological_closing(otsu_image_paths, output_morphological_closing)

    # Step 6: Apply Sobel Edge Detection
    morph_image_paths = load_images_from_folder(output_morphological_closing)
    apply_unsharp_masking(morph_image_paths, output_edge_unsharp)

Main flow but with black on white images ending with sharp

In [ ]:
def process_image_flow_normal_thresh(folder_path, output_noise_gaussian, output_hist_clahe, output_bi_otsu, output_morphological_opening, output_edge_unsharp):
    # Step 1: Load images
    image_paths = load_images_from_folder(folder_path)

    # Step 2: Apply Gaussian Blur
    apply_gaussian_blur(image_paths, output_noise_gaussian)

    # Step 3: Apply CLAHE
    blurred_image_paths = load_images_from_folder(output_noise_gaussian)
    apply_clahe(blurred_image_paths, output_hist_clahe)

    # Step 4: Apply Otsu Thresholding 
    clahe_image_paths = load_images_from_folder(output_hist_clahe)
    apply_otsu_thresholding(clahe_image_paths, output_bi_otsu)

    # Step 5: Apply Morphological Opening
    otsu_image_paths = load_images_from_folder(output_bi_otsu)
    apply_morphological_opening(otsu_image_paths, output_morphological_opening)

    # Step 6: Apply Sobel Edge Detection
    morph_image_paths = load_images_from_folder(output_morphological_opening)
    apply_unsharp_masking(morph_image_paths, output_edge_unsharp)

Flow with just Morphological changes

In [ ]:
def process_image_flow_morph(folder_path, output_noise_gaussian, output_hist_clahe, output_bi_otsu_inv, output_morphological_closing):
    # Step 1: Load images
    image_paths = load_images_from_folder(folder_path)

    # Step 2: Apply Gaussian Blur
    apply_gaussian_blur(image_paths, output_noise_gaussian)

    # Step 3: Apply CLAHE
    blurred_image_paths = load_images_from_folder(output_noise_gaussian)
    apply_clahe(blurred_image_paths, output_hist_clahe)

    # Step 4: Apply Otsu Thresholding Inverted
    clahe_image_paths = load_images_from_folder(output_hist_clahe)
    apply_otsu_thresholding_inv(clahe_image_paths, output_bi_otsu_inv)

    # Step 5: Apply Morphological Closing
    otsu_image_paths = load_images_from_folder(output_bi_otsu_inv)
    apply_morphological_closing(otsu_image_paths, output_morphological_closing)


Flow with just binization

In [ ]:
def process_image_flow_bi(folder_path, output_noise_gaussian, output_hist_clahe, output_bi_otsu_inv):
    # Step 1: Load images
    image_paths = load_images_from_folder(folder_path)

    # Step 2: Apply Gaussian Blur
    apply_gaussian_blur(image_paths, output_noise_gaussian)

    # Step 3: Apply CLAHE
    blurred_image_paths = load_images_from_folder(output_noise_gaussian)
    apply_clahe(blurred_image_paths, output_hist_clahe)

    # Step 4: Apply Otsu Thresholding Inverted
    clahe_image_paths = load_images_from_folder(output_hist_clahe)
    apply_otsu_thresholding_inv(clahe_image_paths, output_bi_otsu_inv)


Flow with just CLAHE

In [ ]:
def process_image_flow_clahe(folder_path, output_noise_gaussian, output_hist_clahe):
    # Step 1: Load images
    image_paths = load_images_from_folder(folder_path)

    # Step 2: Apply Gaussian Blur
    apply_gaussian_blur(image_paths, output_noise_gaussian)

    # Step 3: Apply CLAHE
    blurred_image_paths = load_images_from_folder(output_noise_gaussian)
    apply_clahe(blurred_image_paths, output_hist_clahe)